In [0]:
"""
既に元コードで生成された10個の `.xyz` ファイル（例: `pipe_case_00.xyz`, `pipe_case_01.xyz`, ...）がある前提で、
- それらのファイルを読み込み、
- 点群データから各断面の円周点数を推定（または指定）して、
- STLメッシュに変換して書き出す、
コードを示します。
---
### 変更方針
- `.xyz` ファイルは単に頂点座標が並んだテキストなので、`np.loadtxt` で読み込み可能。
- 点群は連続した断面が `num_circ_points` ごとに区切られている想定。
- `num_circ_points` は元のパイプ生成時と同じ値を使う必要があります。
- 10個分のファイルを読み込んでSTLに変換する処理をまとめた関数を作成。
---
### コード例
"""
import numpy as np
import os

def load_xyz_files(directory="data/point_cloud", file_pattern="pipe_case_{:02d}.xyz", n_files=10):
    """
    指定ディレクトリ内の複数xyzファイルを読み込み、リストで返す。
    """
    pointclouds = []
    for i in range(n_files):
        filename = os.path.join(directory, file_pattern.format(i))
        if not os.path.exists(filename):
            raise FileNotFoundError(f"ファイルが見つかりません: {filename}")
        # ヘッダー行を飛ばすため skiprows=1 を追加
        pts = np.loadtxt(filename, skiprows=1)
        if pts.ndim != 2 or pts.shape[1] != 3:
            raise ValueError(f"{filename} のフォーマットが不正です。shape={pts.shape}")
        pointclouds.append(pts)
        print(f"Loaded {filename}, points: {pts.shape}")
    return pointclouds
def build_stl_from_pipe_points(pipe_points_list, num_circ_points, output_directory="data/stl", filename_prefix="pipe_case"):
    """
    pipe_points_list: 各ケースの (N,3) 点群（円断面連続）
    num_circ_points: 円断面の点数（円周方向点数）
    output_directory: STL出力先ディレクトリ
    filename_prefix: ファイル名の接頭辞
    """
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)
    for case_idx, pts in enumerate(pipe_points_list):
        if pts.shape[0] % num_circ_points != 0:
            raise ValueError(f"点数が断面数×円周点数になっていません。pts.shape={pts.shape}, num_circ_points={num_circ_points}")
        n_sections = pts.shape[0] // num_circ_points
        triangles = []
        for i in range(n_sections - 1):
            base_idx = i * num_circ_points
            next_base_idx = (i+1) * num_circ_points
            for j in range(num_circ_points):
                j_next = (j+1) % num_circ_points
                v0 = pts[base_idx + j]
                v1 = pts[base_idx + j_next]
                v2 = pts[next_base_idx + j_next]
                v3 = pts[next_base_idx + j]
                triangles.append((v0, v1, v2))
                triangles.append((v0, v2, v3))
        filename = os.path.join(output_directory, f"{filename_prefix}_{case_idx:02d}.stl")
        write_ascii_stl(filename, triangles)
        print(f"STL saved: {filename}")
def write_ascii_stl(filename, triangles):
    with open(filename, "w") as f:
        f.write("solid pipe\n")
        for tri in triangles:
            v0, v1, v2 = tri
            normal = np.cross(v1 - v0, v2 - v0)
            n_norm = np.linalg.norm(normal)
            if n_norm < 1e-12:
                normal = np.array([0.0, 0.0, 0.0])
            else:
                normal = normal / n_norm
            f.write(f"  facet normal {normal[0]:.6e} {normal[1]:.6e} {normal[2]:.6e}\n")
            f.write("    outer loop\n")
            f.write(f"      vertex {v0[0]:.6e} {v0[1]:.6e} {v0[2]:.6e}\n")
            f.write(f"      vertex {v1[0]:.6e} {v1[1]:.6e} {v1[2]:.6e}\n")
            f.write(f"      vertex {v2[0]:.6e} {v2[1]:.6e} {v2[2]:.6e}\n")
            f.write("    endloop\n")
            f.write("  endfacet\n")
        f.write("endsolid pipe\n")
if __name__ == "__main__":
    pipe_radius = 0.05
    target_spacing = 0.02
    # 元コードと同じ計算で円周点数を推定
    num_circ_points = max(8, int(round(2.0*np.pi * pipe_radius / target_spacing)))
    # xyzファイルを読み込み（data/point_cloudフォルダ内）
    pointclouds = load_xyz_files(directory="data/point_cloud", file_pattern="pipe_case_{:02d}.xyz", n_files=10)
    # STLに変換して data/stl フォルダに保存
    build_stl_from_pipe_points(pointclouds, num_circ_points, output_directory="data/stl", filename_prefix="pipe_case")

"""
---
### 使い方
- 元コードで生成した `pipe_case_00.xyz` ～ `pipe_case_09.xyz` が同じフォルダにある状態で上記スクリプトを実行してください。
- それぞれ対応する `pipe_case_00.stl` ～ `pipe_case_09.stl` が生成されます。
---
必要に応じてパスやファイル名のパターン、`pipe_radius` と `target_spacing` を元コードと合わせて調整してください。
"""


In [0]:
# 必要ライブラリのインストール（Databricksではセル先頭の %pip が使えます）
# 実行環境によっては既にインストール済みの場合があります
%pip install numpy-stl plotly
import numpy as np
from stl import mesh
import plotly.graph_objects as go
import os
# ========== 等スケール補助 ==========
def compute_equal_ranges(points, pad_ratio=0.05):
    """
    points: (N,3)
    3軸の表示範囲を同一幅に揃えるため、中心と最大幅から range を計算
    """
    mins = points.min(axis=0)
    maxs = points.max(axis=0)
    center = 0.5 * (mins + maxs)
    max_span = float(np.max(maxs - mins))
    half = 0.5 * max_span * (1.0 + pad_ratio)
    xr = [center[0] - half, center[0] + half]
    yr = [center[1] - half, center[1] + half]
    zr = [center[2] - half, center[2] + half]
    return xr, yr, zr
def apply_equal_scale_plotly(fig, points, pad_ratio=0.05):
    """
    Plotlyの3Dシーンに等スケール（立方体表示）を適用
    """
    xr, yr, zr = compute_equal_ranges(points, pad_ratio=pad_ratio)
    fig.update_layout(
        scene=dict(
            xaxis=dict(range=xr, title='X'),
            yaxis=dict(range=yr, title='Y'),
            zaxis=dict(range=zr, title='Z'),
            aspectmode='cube'  # 立方体で表示（rangeを揃えているので同一スケール）
        )
    )
# ========== STL読み込み ==========
def load_stl_mesh(filename):
    if not os.path.exists(filename):
        raise FileNotFoundError(f"File not found: {filename}")
    return mesh.Mesh.from_file(filename)
# ========== メッシュ＆法線のPlotlyトレース作成 ==========
def build_mesh_and_normals_traces(stl_mesh, normal_stride=2, normal_scale=0.08,
                                  mesh_color='lightblue', mesh_opacity=0.7, normal_color='red'):
    """
    - Mesh3d（メッシュ）と Cone（法線矢印）を作成
    - normal_stride: 法線の間引きレート（表示本数を減らして軽量化）
    - normal_scale : 法線矢印サイズ
    """
    tri = stl_mesh.vectors            # (n_tri, 3, 3)
    vertices = tri.reshape(-1, 3)     # (n_tri*3, 3)
    x, y, z = vertices.T
    i = np.arange(0, len(x), 3)
    j = i + 1
    k = i + 2
    mesh3d = go.Mesh3d(
        x=x, y=y, z=z, i=i, j=j, k=k,
        color=mesh_color, opacity=mesh_opacity,
        flatshading=True, name="mesh"
    )
    # 法線（各三角形の重心にCone）
    centroids = tri.mean(axis=1)      # (n_tri, 3)
    normals = stl_mesh.normals        # (n_tri, 3)
    idx = np.arange(tri.shape[0])[::max(1, int(normal_stride))]
    C = centroids[idx]
    N = normals[idx]
    cones = go.Cone(
        x=C[:, 0], y=C[:, 1], z=C[:, 2],
        u=N[:, 0], v=N[:, 1], w=N[:, 2],
        anchor="tail",
        sizemode="absolute", sizeref=normal_scale,
        colorscale=[[0, normal_color], [1, normal_color]],
        showscale=False, name="normals"
    )
    return mesh3d, cones, vertices
# ========== 単一ファイル表示 ==========
def show_single_case(stl_dir='/dbfs/data/stl', filename='pipe_case_00.stl',
                     normal_stride=2, normal_scale=0.08):
    path = os.path.join(stl_dir, filename) if not filename.startswith('/') else filename
    stl_mesh = load_stl_mesh(path)
    mesh3d, cones, vertices = build_mesh_and_normals_traces(
        stl_mesh, normal_stride=normal_stride, normal_scale=normal_scale
    )
    fig = go.Figure([mesh3d, cones])
    apply_equal_scale_plotly(fig, vertices, pad_ratio=0.05)
    fig.update_layout(
        margin=dict(l=0, r=0, t=30, b=0),
        title=f"STL Mesh with Normals - {os.path.basename(path)}"
    )
    fig.show()
# ========== 複数ファイルをドロップダウンで切替 ==========
def show_dropdown_cases(stl_dir='/dbfs/data/stl', pattern='pipe_case_{:02d}.stl', n_files=10,
                        normal_stride=2, normal_scale=0.08):
    fig = go.Figure()
    all_vertices = []
    paths = [os.path.join(stl_dir, pattern.format(i)) for i in range(n_files)]
    # 各ケースのトレースを追加（mesh + normals）
    for p in paths:
        if not os.path.exists(p):
            # 空トレースでプレースホルダ
            fig.add_trace(go.Mesh3d(x=[], y=[], z=[], i=[], j=[], k=[], name="mesh"))
            fig.add_trace(go.Cone(x=[], y=[], z=[], u=[], v=[], w=[], name="normals"))
        else:
            stl_mesh = load_stl_mesh(p)
            mesh3d, cones, vertices = build_mesh_and_normals_traces(
                stl_mesh, normal_stride=normal_stride, normal_scale=normal_scale
            )
            fig.add_trace(mesh3d)
            fig.add_trace(cones)
            all_vertices.append(vertices)
    # 初期可視状態（Case 0のみ表示）
    visible = [False] * (2 * n_files)
    if n_files > 0:
        visible[0] = True  # mesh
        visible[1] = True  # normals
    for idx, v in enumerate(visible):
        fig.data[idx].visible = v
    # ドロップダウンボタン
    buttons = []
    for i in range(n_files):
        vis = [False] * (2 * n_files)
        vis[2*i] = True
        vis[2*i + 1] = True
        buttons.append(dict(
            label=f"Case {i}",
            method="update",
            args=[{"visible": vis},
                  {"title": f"STL Mesh with Normals - Case {i}"}]
        ))
    # 全ケースの頂点から等スケール範囲を計算して一括適用（切替時もスケール統一）
    V = np.vstack(all_vertices) if len(all_vertices) > 0 else np.zeros((1, 3))
    apply_equal_scale_plotly(fig, V, pad_ratio=0.05)
    fig.update_layout(
        updatemenus=[dict(
            type="dropdown",
            buttons=buttons,
            x=1.0, xanchor="right",
            y=1.0, yanchor="top"
        )],
        margin=dict(l=0, r=0, t=30, b=0),
        title="STL Mesh with Normals - Case 0"
    )
    fig.show()
# ========== 実行設定 ==========
MODE = 'dropdown'  # 'single'（単一ファイル） or 'dropdown'（複数切替）
STL_DIR = 'data/stl'
N_FILES = 10
NORMAL_STRIDE = 2   # 法線間引き（2,5,10などで軽量化）
NORMAL_SCALE = 0.08 # 法線矢印サイズ（モデル寸法に合わせて調整）
if MODE == 'single':
    # 単一ファイル表示
    show_single_case(stl_dir=STL_DIR, filename='pipe_case_00.stl',
                     normal_stride=NORMAL_STRIDE, normal_scale=NORMAL_SCALE)
else:
    # ドロップダウン切替表示
    show_dropdown_cases(stl_dir=STL_DIR, pattern='pipe_case_{:02d}.stl', n_files=N_FILES,
                        normal_stride=NORMAL_STRIDE, normal_scale=NORMAL_SCALE)